In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import tensorflow_addons
from gensim.models import KeyedVectors
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [38]:
from tensorflow.keras import Model
from tensorflow_addons.text import crf_log_likelihood, viterbi_decode, crf_decode
from tensorflow.keras.layers import Input, LSTM, Embedding, Bidirectional, Masking, Dense, Layer, TimeDistributed, Activation, Lambda, Dropout, InputSpec
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.layers import Conv1D

In [20]:
from transformers import TFBertModel

In [3]:
tf.keras.backend.clear_session()

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-02-14 11:02:21.752218: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-14 11:02:21.828420: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-14 11:02:21.890527: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-14 11:02:21.890780: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

True

## 数据准备

In [5]:
train_set = pd.read_csv('data/msr_training.utf8', encoding= 'utf8', header=None)  # 不把第一行作为列属性，且pd读出来就是数据帧，就是字符串
test_set = pd.read_csv('data/msr_test_gold.utf8', encoding='utf8', header=None)
print(train_set.head())
print(test_set.head())

                                                   0
0  “  人们  常  说  生活  是  一  部  教科书  ，  而  血  与  火  ...
1  “  心  静  渐  知  春  似  海  ，  花  深  每  觉  影  生  香  。
2    “  吃  屎  的  东西  ，  连  一  捆  麦  也  铡  不  动  呀  ？
3  他  “  严格要求  自己  ，  从  一个  科举  出身  的  进士  成为  一...
4  “  征  而  未  用  的  耕地  和  有  收益  的  土地  ，  不准  ...
                                                   0
0                      扬帆  远东  做  与  中国  合作  的  先行  
1                            希腊  的  经济  结构  较  特殊  。
2  海运  业  雄踞  全球  之  首  ，  按  吨位  计  占  世界  总数  的...
3  另外  旅游  、  侨汇  也是  经济  收入  的  重要  组成部分  ，  制造业...
4  多年来  ，  中  希  贸易  始终  处于  较低  的  水平  ，  希腊  几乎...


In [6]:
# 将句子转换成字序列
def get_char(sentence):
    char_list = []
    sentence = ''.join(sentence.split('  ')) #去掉空格
    for i in sentence:
        char_list.append(i)
    return char_list

In [7]:
#将句子转成BMES序列
def get_label(sentence):
    result = []
    word_list = sentence.split('  ')  #两个空格来分隔一个词
    for i in range(len(word_list)):
        if len(word_list[i]) == 1:
            result.append('S')
        elif len(word_list[i]) == 2:
            result.append('B')
            result.append('E')
        else:
            temp = len(word_list[i]) - 2
            result.append('B')
            result.extend('M'*temp)
            result.append('E')
    return result

In [8]:
def read_file(file):
    char, content, label = [], [], []
    maxlen = 0

    for i in range(len(file)):  # 记得加range！！
        line = file.loc[i,0]   # 用loc来访问dataframe
        line = line.strip('\n') #去掉换行符
        line = line.strip(' ')  #去掉开头和结尾的空格
        
        char_list = get_char(line)        #获得字列表
        label_list = get_label(line)      # 获得标签列表
        maxlen = max(maxlen, len(char_list))
        if len(char_list)!=len(label_list):
            continue   # 由于数据集的问题，所以要删掉有问题的样本（在训练集中有26个样本；测试集中无）
        char.extend(char_list)            #每一个单元是1个字
        content.append(char_list)         # 每一个单元是一行里面的各个字（分好）
        label.append(label_list)          #每一个单元是一行里面打好标签的结果（含标点）
    return char, content, label, maxlen  #word是单列表，content和label是双层列表


In [9]:
# process data: padding
def process_data(char_list, label_list, vocab, chunk_tags, MAXLEN):
    # idx2vocab = {idx: char for idx, char in enumerate(vocab)}
    vocab2idx = {char: idx for idx, char in enumerate(vocab)}
    # get every char of every word, map to idx in vocab, set to <UNK> if not in vocab
    x = [[vocab2idx.get(char, 1) for char in s] for s in char_list]
    # map label to idx
    y_chunk = [[chunk_tags.index(label) for label in s] for s in label_list]
    # padding of x, default is 0(symbolizes <PAD>). padding includes:over->cutoff, less->padding. default: left_padding
    x = pad_sequences(x, maxlen=MAXLEN, value=0)
    # padding of y_chunk
    y_chunk = pad_sequences(y_chunk, maxlen=MAXLEN, value=-1)
    # one_hot:
    y_chunk = to_categorical(y_chunk, len(chunk_tags))
    # y_chunk = np.eye(len(chunk_tags), dtype='float32')[y_chunk]
    return x, y_chunk

In [10]:
def load_data():
    chunk_tags = ['S','B','M','E']
    train_char, train_content, train_label, _ = read_file(train_set)
    test_char, test_content, test_label, maxlen = read_file(test_set)
    
    vocab = list(set(train_char + test_char))   # 合并，构成大词表
    special_chars = ['<PAD>', '<UNK>']   #特殊词表示：PAD表示padding，UNK表示词表中没有
    vocab = special_chars + vocab
    
    # save initial config data
#     with open(SAVE_PATH, 'wb') as f:
#         pickle.dump((train_char, chunk_tags), f)
    
    # process data: padding
    print('maxlen is %d' % maxlen)
    train_x, train_y = process_data(train_content, train_label, vocab, chunk_tags, maxlen)
    test_x, test_y = process_data(test_content, test_label, vocab, chunk_tags, maxlen)
    return train_x, train_y, test_x, test_y, vocab, chunk_tags, maxlen, test_content

In [11]:
train_x, train_y, test_x, test_y, vocab, chunk_tags, max_len, test_content = load_data()

n_words = len(vocab)
n_tags = len(chunk_tags)

maxlen is 308


## 构建网络

In [22]:
def make_embeddings_matrix(word2vec_model, vocab):
    char2vec_dict = {}    # 字对词向量
    vocab2idx = {char: idx for idx, char in enumerate(vocab)}
    for char, vector in zip(word2vec_model.vocab, word2vec_model.vectors):
        char2vec_dict[char] = vector
    embeddings_matrix = np.zeros((len(vocab), EMBED_DIM))# form huge matrix
    for i in tqdm(range(2, len(vocab))):
        char = vocab[i]
        if char in char2vec_dict.keys():    # 如果char在词向量列表中，更新权重；否则，赋值为全0（默认）
            char_vector = char2vec_dict[char]
            embeddings_matrix[i] = char_vector
    return embeddings_matrix

In [18]:
class AttentionSelf(Layer):
    """
        self attention,
        codes from:  https://mp.weixin.qq.com/s/qmJnyFMkXVjYBwoR_AQLVA
    """
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super().__init__(**kwargs)

    def build(self, input_shape):
        # W、K and V
        self.kernel = self.add_weight(name='WKV',
                                        shape=(3, input_shape[2], self.output_dim),
                                        initializer='uniform',
                                        regularizer=L1L2(0.0000032),
                                        trainable=True)
        super().build(input_shape)

    def call(self, x):
        WQ = K.dot(x, self.kernel[0])
        WK = K.dot(x, self.kernel[1])
        WV = K.dot(x, self.kernel[2])
        print("WQ.shape",WQ.shape)
        print("K.permute_dimensions(WK, [0, 2, 1]).shape",K.permute_dimensions(WK, [0, 2, 1]).shape)
        QK = K.batch_dot(WQ,K.permute_dimensions(WK, [0, 2, 1]))
        QK = QK / (100**0.5)
        QK = K.softmax(QK)
        print("QK.shape",QK.shape)
        V = K.batch_dot(QK,WV)
        return V

    def compute_output_shape(self, input_shape):
        return (input_shape[0],input_shape[1],self.output_dim)

In [63]:
def embedding_layer(input_dim, output_dim, input_length, mask_zero):
    return Embedding(input_dim = input_dim, output_dim = output_dim, input_length = input_length, mask_zero = mask_zero)
    
def bilstm_crf(maxlen, n_tags, embedding_dim, n_words, mask_zero, training = True):
    """
    bilstm_crf - module to build BiLSTM-CRF model
    Inputs:
        - input_shape : tuple
            Tensor shape of inputs, excluding batch size
    Outputs:
        - output : tensorflow.keras.outputs.output
            BiLSTM-CRF output
    """
    input = Input(shape = (maxlen,))
    
    # Embedding layer
    embeddings = embedding_layer(input_dim=n_words, output_dim=embedding_dim, input_length=maxlen, mask_zero=mask_zero)
    output = embeddings(input)

    # BiLSTM layer
    output = Bidirectional(LSTM(units=50, return_sequences=True, recurrent_activation = 'sigmoid', recurrent_dropout=0, activation='tanh'))(output)
#     output = AttentionSelf(200)(output)

    # Dense layer
    output = TimeDistributed(Dense(n_tags, activation='relu'))(output)
    
    output = CRF(n_tags, name='crf_layer')(output)
    return Model(input, output)


class CRF(Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 **kwargs):
        """    
        Args:
            output_dim (int): the number of labels to tag each temporal input.
            sparse_target (bool): whether the the ground-truth label represented in one-hot.
        Input shape:
            (batch_size, sentence length, output_dim)
        Output shape:
            (batch_size, sentence length, output_dim)
        """
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim) 
        self.sparse_target = sparse_target
        self.input_spec = InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = None

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` '
                             'should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output'
                             ' shape. Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        # Just pass the received mask from previous layer, to the next layer or
        # manipulate it if this layer changes the shape of the input
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            # -1e10 to avoid zero at sum(mask)
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'supports_masking': self.supports_masking,
            'transitions': K.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()
        return dict(base_config, **config)

In [67]:
model = bilstm_crf(maxlen=max_len, n_tags=n_tags, embedding_dim=20, n_words=n_words, mask_zero=True)
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 308)]             0         
                                                                 
 embedding_8 (Embedding)     (None, 308, 20)           103620    
                                                                 
 bidirectional_5 (Bidirectio  (None, 308, 100)         28400     
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 308, 4)           404       
 tributed)                                                       
                                                                 
 crf_layer (CRF)             (None, 308, 4)            16        
                                                                 
Total params: 132,440
Trainable params: 132,440
Non-trainab

In [68]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer = Adam(learning_rate = 0.01), loss = model.layers[-1].loss, metrics = model.layers[-1].accuracy)

## 模型训练

In [69]:
history = model.fit(train_x, train_y, batch_size=32, epochs=1, verbose=1, validation_split=0.1)
print(history.history)

2444/2444 [==============================] - 1483s 604ms/step - loss: 2.4383 - viterbi_accuracy: 0.9810 - val_loss: 67.4596 - val_viterbi_accuracy: 0.8943
{'loss': [2.4382832050323486], 'viterbi_accuracy': [0.9809593558311462], 'val_loss': [67.45957946777344], 'val_viterbi_accuracy': [0.8942558169364929]}


### IDCNN-CRF

In [42]:
def bilstm_crf(maxlen, n_tags, embedding_dim, n_words, mask_zero, training = True):
    """
    bilstm_crf - module to build BiLSTM-CRF model
    Inputs:
        - input_shape : tuple
            Tensor shape of inputs, excluding batch size
    Outputs:
        - output : tensorflow.keras.outputs.output
            BiLSTM-CRF output
    """
    input = Input(shape = (maxlen,))
    
    # Embedding layer
    embeddings = embedding_layer(input_dim=n_words, output_dim=embedding_dim, input_length=maxlen, mask_zero=mask_zero)
    output = embeddings(input)

    # BiLSTM layer
    output = Conv1D(filters=64,
                    kernel_size=3,
                    activation='relu',
                    padding='same',
                    dilation_rate=1)(output)
    
    output = Conv1D(filters=128,
                    kernel_size=3,
                    activation='relu',
                    padding='same',
                    dilation_rate=1)(output)
    
    output = Conv1D(filters=128,
                    kernel_size=3,
                    activation='relu',
                    padding='same',
                    dilation_rate=2)(output)
    
    
    # Dense layer
    output = Dense(n_tags)(output)
    
    output = CRF(n_tags, name='crf_layer')(output)
    return Model(input, output)

model = bilstm_crf(maxlen=max_len, n_tags=n_tags, embedding_dim=20, n_words=n_words, mask_zero=True)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 308)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 308, 20)           103620    
                                                                 
 conv1d_3 (Conv1D)           (None, 308, 64)           3904      
                                                                 
 conv1d_4 (Conv1D)           (None, 308, 128)          24704     
                                                                 
 conv1d_5 (Conv1D)           (None, 308, 128)          49280     
                                                                 
 dense_3 (Dense)             (None, 308, 4)            516       
                                                                 
 crf_layer (CRF)             (None, 308, 4)            16  

## 模型测试

In [45]:
score = model.evaluate(test_x, test_y, batch_size=256)
print(score)

16/16 [==============================] - 3s 158ms/step - loss: 941.2169 - viterbi_accuracy: 0.5467
[941.2169189453125, 0.5467380881309509]


In [70]:
test_predict = model.predict(test_x)
test_predict = [[np.argmax(char) for char in sample] for sample in test_predict]  # get the max label_id
test_predict_tag = [[chunk_tags[i] for i in sample ]for sample in test_predict]   # get the label of predic
test_gold = [[np.argmax(char) for char in sample] for sample in test_y]  # get the label_id
test_gold_tag = [[chunk_tags[i] for i in sample] for sample in test_gold]  # get the label of real

In [71]:
test_result = []
vocab2idx = {char: idx for idx, char in enumerate(vocab)}
_, test_content, _, _ = read_file(test_set)
for i in range(len(test_predict)):
# for i in range(1):
    sentence = ''
    s_len = len(test_content[i])
    sample = test_predict_tag[i]
    for j in range(s_len):
        idx = len(sample)- s_len + j
        if sample[idx]=='B' or sample[idx]=='M' or j==s_len-1:
            sentence = sentence + test_content[i][j]
        else:
            sentence = sentence + test_content[i][j]
            sentence = sentence + '  '
    test_result.append(sentence)
print('\n'.join(test_result[:5]))

扬帆远东  做  与  中国  合作  的  先行
希腊  的  经济  结构  较  特殊  。
海运业  雄踞  全球  之  首  ，  按  吨位  计占  世界  总数  的  １７％  。
另外  旅游  、  侨汇  也是  经济  收入  的  重要  组成部分  ，  制造业  规  模  相  对  较小  。
多年  来  ，  中希贸易  始终  处于  较低  的  水平  ，  希腊  几乎  没有  在  中国  投资  。


In [47]:
test_result = []
vocab2idx = {char: idx for idx, char in enumerate(vocab)}
_, test_content, _, _ = read_file(test_set)
for i in range(len(test_predict)):
# for i in range(1):
    sentence = ''
    s_len = len(test_content[i])
    sample = test_predict_tag[i]
    for j in range(s_len):
        idx = len(sample)- s_len + j
        if sample[idx]=='B' or sample[idx]=='M' or j==s_len-1:
            sentence = sentence + test_content[i][j]
        else:
            sentence = sentence + test_content[i][j]
            sentence = sentence + '  '
    test_result.append(sentence)
print('\n'.join(test_result[:5]))

扬帆  远东  做  与  中国  合作  的  先  行
希腊  的  经济  结构  较  特殊  。
海运业  雄踞  全球  之  首  ，  按  吨  位计  占  世界  总数  的  １７％  。
另外  旅游  、  侨汇  也是  经济  收入  的  重要  组成部分  ，  制造业  规模  相对  较小  。
多年来  ，  中希  贸易  始终  处于  较低  的  水平  ，  希腊  几乎  没有  在  中国  投资  。


## 模型保存

In [20]:
tf.keras.models.save_model(model, filepath = 'bilstm_crf', save_format = 'tf')

2022-02-09 16:50:59.131867: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: bilstm_crf/assets


In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('bilstm_crf.tflite', 'wb') as f:
    f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmpupy68hk7/assets


In [22]:
!ls -lh

总用量 569M
-rw-rw-r-- 1 huanfu huanfu 713K 2月   7 14:55 期中作业.pdf
-rw-rw-r-- 1 huanfu huanfu  25K 2月   7 14:55 中文分词实验报告.md
drwxr-xr-x 4 huanfu huanfu 4.0K 2月   8 16:56 bilstm_crf
-rw-rw-r-- 1 huanfu huanfu 695K 2月   8 17:59 bilstm_crf.tflite
drwxrwxr-x 3 huanfu huanfu 4.0K 2月   7 15:57 code
drwxrwxr-x 2 huanfu huanfu 4.0K 2月   7 14:55 data
-rw-rw-r-- 1 huanfu huanfu  32K 2月   8 15:13 main.ipynb
-rw-rw-r-- 1 huanfu huanfu  11K 2月   7 16:02 main.py
-rw-rw-r-- 1 huanfu huanfu 7.2M 2月   7 18:27 model.h5
-rw-rw-r-- 1 huanfu huanfu 2.0K 2月   7 14:55 README.md
-rw-rw-r-- 1 huanfu huanfu 1.5M 2月   7 14:55 report.pdf
-rwxr----- 1 huanfu huanfu 559M 2月   7 17:04 sgns.context.word-character.char1-1.bz2
-rw-rw-r-- 1 huanfu huanfu  24K 2月   8 18:00 wordseg.ipynb


## 词汇表和tag打包

In [25]:
with open("vocabs.txt", "w") as f:
    for key, val in vocab2idx.items():
        f.write("%s %d\n" % (key, val))
        
with open("tags.txt", "w") as f:
    for tag in chunk_tags:
        f.write("%s\n" % tag)

In [26]:
from tflite_support import metadata as _metadata

populator = _metadata.MetadataPopulator.with_model_file("bilstm_crf.tflite")
populator.load_associated_files(["vocabs.txt","tags.txt"])

populator.populate()

/home/huanfu/miniconda3/envs/tf2/lib/python3.6/site-packages/tensorflow_lite_support/metadata/python/metadata.py:359: UserWarning: File, 'tags.txt', does not exist in the metadata. But packing it to tflite model is still allowed.
  "tflite model is still allowed.".format(f))
/home/huanfu/miniconda3/envs/tf2/lib/python3.6/site-packages/tensorflow_lite_support/metadata/python/metadata.py:359: UserWarning: File, 'vocabs.txt', does not exist in the metadata. But packing it to tflite model is still allowed.
  "tflite model is still allowed.".format(f))


In [36]:
!ls

期中作业.pdf	     data	 report.pdf
中文分词实验报告.md  main.ipynb  sgns.context.word-character.char1-1.bz2
bilstm_crf	     main.py	 tags.txt
bilstm_crf.tflite    model.h5	 vocabs.txt
code		     README.md	 wordseg.ipynb


## 测试tflite

In [48]:
interpreter = tf.lite.Interpreter(model_path="bilstm_crf.tflite")

In [49]:
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print("")
print(output_details)

[{'name': 'input_1', 'index': 0, 'shape': array([  1, 308], dtype=int32), 'shape_signature': array([ -1, 308], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

[{'name': 'Identity', 'index': 204, 'shape': array([  1, 308,   4], dtype=int32), 'shape_signature': array([  1, 308,   4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [50]:
len(output_details)

1

In [57]:
input_data = test_x[:1].astype(np.float32)
input_data.shape

(1, 308)

In [58]:
index = input_details[0]['index']
interpreter.set_tensor(index, input_data)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data.shape)

(1, 308, 4)


In [ ]:
希腊  的  经济  结构  较  特殊  。